# Mount and Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/My Drive/Few-Shot Project/SSL_ALPNet/

/content/drive/.shortcut-targets-by-id/1kJ6hvD4S-gF34kZ670rDNwU-KvWnQSXg/Few-Shot Project/SSL_ALPNet


In [3]:
!pip install SimpleITK -q
!pip install sacred==0.7.5

     |████████████████████████████████| 48.4 MB 30 kB/s 
     |████████████████████████████████| 92 kB 270 kB/s 
     |████████████████████████████████| 99 kB 9.9 MB/s 
     |████████████████████████████████| 67 kB 6.1 MB/s 
  Created wheel for jsonpickle: filename=jsonpickle-0.9.6-py3-none-any.whl size=29479 sha256=905d6f300dfed21826a8fba75f6b2922d32a31de8566365cebc3360acded58eb
  Stored in directory: /root/.cache/pip/wheels/00/ca/bb/3f8f249a9caf7590412d0c0abe1122a809a46f6930751bb50b
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=8db24d3a2efa6976723e8dcb604ee0f38e3d77eea7363b4819f95c88944857fd
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
Successfully built jsonpickle py-cpuinfo


# Train

In [ ]:
%%shell

# train a model to segment abdominal MRI (T2 fold of CHAOS challenge)
GPUID1=0
export CUDA_VISIBLE_DEVICES=$GPUID1

####### Shared configs
PROTO_GRID=8 # using 32 / 8 = 4, 4-by-4 prototype pooling window during training
CPT="myexperiments"
DATASET='CHAOST2_Superpix'
NWORKER=4

ALL_EV=(0) # 5-fold cross validation (0, 1, 2, 3, 4)
ALL_SCALE=("MIDDLE") # config of pseudo labels

LABEL_SETS=0
EXCLU='[2,3]' # setting 2: excluding kidneys in training set to test generalization capability even though they are unlabeled. Use [] for setting 1 by Roy et al.

### Use Liver and spleen as testing classes
# LABEL_SETS=1
# EXCLU='[1,4]'

NSTEP=100100
DECAY=0.95
MAX_ITER=1000 # defines the size of an epoch
SNAPSHOT_INTERVAL=10000  # interval for saving snapshot
SEED='1234'

supix_matching_threshold=0.7
pre_trained_folder='None'
pre_trained_snapshot='None'
saving_root='./'

###### Validation configs ######
SUPP_ID='[4]' #  # using the additionally loaded scan as support

echo ===================================

for EVAL_FOLD in "${ALL_EV[@]}"
do
    for SUPERPIX_SCALE in "${ALL_SCALE[@]}"
    do
    PREFIX="train_${DATASET}_lbgroup${LABEL_SETS}_scale_${SUPERPIX_SCALE}_vfold${EVAL_FOLD}"
    echo $PREFIX
    LOGDIR="./exps/${CPT}_${SUPERPIX_SCALE}_${LABEL_SETS}"

    if [ ! -d $LOGDIR ]
    then
        mkdir $LOGDIR
    fi

    python3 training.py with \
    'modelname=dlfcn_res101' \
    'usealign=True' \
    'optim_type=sgd' \
    num_workers=$NWORKER \
    scan_per_load=-1 \
    label_sets=$LABEL_SETS \
    'use_wce=True' \
    exp_prefix=$PREFIX \
    'clsname=grid_proto' \
    n_steps=$NSTEP \
    exclude_cls_list=$EXCLU \
    eval_fold=$EVAL_FOLD \
    dataset=$DATASET \
    proto_grid_size=$PROTO_GRID \
    max_iters_per_load=$MAX_ITER \
    min_fg_data=1 seed=$SEED \
    save_snapshot_every=$SNAPSHOT_INTERVAL \
    superpix_scale=$SUPERPIX_SCALE \
    lr_step_gamma=$DECAY \
    path.log_dir=$LOGDIR \
    support_idx=$SUPP_ID \
    supix_matching_threshold=$supix_matching_threshold \
    'create_supix_matching_prep_file=False' \
    'use_supix_matching=False' \
    'exclude_testing_objs=True' \
    'use_pre_trained=False' \
    pre_trained_folder=$pre_trained_folder \
    pre_trained_snapshot=$pre_trained_snapshot \
    saving_root=$saving_root
    done
done


train_CHAOST2_Superpix_lbgroup0_scale_MIDDLE_vfold0
WARNING - root - Changed type of config entry "min_fg_data" from str to int
WARNING - root - Changed type of config entry "pre_trained_folder" from int to NoneType
WARNING - root - Changed type of config entry "pre_trained_snapshot" from int to NoneType
INFO - mySSL - Running command 'main'
INFO - mySSL - Started run with ID "1"
INFO - main - ###### Create model ######
###### NETWORK: Using ms-coco initialization ######
INFO - main - ###### Load data ######
INFO - main - ###### Labels excluded in training : [2, 3] ######
INFO - main - ###### Unseen labels evaluated in testing: [2, 3] ######
Dataset: the following classes has been excluded [2, 3]
Initial scans loaded: 
['10', '13', '15', '19', '20', '21', '22', '31', '32', '33', '34', '36', '37', '38', '39']
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker 

# Test

In [ ]:
%%shell

model_folder="default"
#model_snapshot=60000
reload_root='./'
saving_root='./'

for SNAP in `seq 5000 5000 100000`
do
  python3.7 validation.py with \
  reload_root=$reload_root \
  saving_root=$saving_root \
  reload_base_path='exps/myexperiments_MIDDLE_0/mySSL_train_CHAOST2_Superpix_lbgroup0_scale_MIDDLE_vfold0_CHAOST2_Superpix_sets_0_1shot' \
  model_folder=$model_folder \
  model_snapshot=$SNAP \
  'modelname=dlfcn_res101' \
  'usealign=True' \
  'optim_type=sgd' \
  num_workers=4 \
  scan_per_load=-1 \
  label_sets=0 \
  'use_wce=True' \
  exp_prefix='test_vfold' \
  'clsname=grid_proto' \
  n_steps=4000 \
  exclude_cls_list='[2,3]' \
  eval_fold=0 \
  dataset='CHAOST2_Superpix' \
  proto_grid_size=8 \
  max_iters_per_load=1000 \
  min_fg_data=1 seed='1234' \
  save_snapshot_every=25000 \
  superpix_scale='MIDDLE' \
  lr_step_gamma=0.95 \
  path.log_dir='./exps/myexperiments_MIDDLE_0' \
  support_idx='[4]'

  printf "\n ====== snapshot ${SNAP} completed ======\n"
done

In [4]:
%%shell

model_folder='default'
model_snapshot=20000
reload_root='./'

python3 validation.py with \
reload_root=$reload_root \
reload_base_path='exps/myexperiments_MIDDLE_0/mySSL_train_CHAOST2_Superpix_lbgroup0_scale_MIDDLE_vfold0_CHAOST2_Superpix_sets_0_1shot' \
model_folder=$model_folder \
model_snapshot=$model_snapshot \
'modelname=dlfcn_res101' \
'usealign=True' \
'optim_type=sgd' \
num_workers=4 \
scan_per_load=-1 \
label_sets=0 \
'use_wce=True' \
exp_prefix='test_vfold' \
'clsname=grid_proto' \
n_steps=4000 \
exclude_cls_list='[2,3]' \
eval_fold=0 \
dataset='CHAOST2_Superpix' \
proto_grid_size=8 \
max_iters_per_load=1000 \
min_fg_data=1 seed='1234' \
save_snapshot_every=25000 \
superpix_scale='MIDDLE' \
lr_step_gamma=0.95 \
# path.log_dir='./exps' \
support_idx='[4]'

WARNING - root - Changed type of config entry "min_fg_data" from str to int
WARNING - root - Changed type of config entry "model_folder" from int to str
INFO - mySSL - Running command 'main'
INFO - mySSL - Started run with ID "1"
INFO - main - ###### Reload model from folder/snapshot:  default/20000.pth ######
###### NETWORK: Using ms-coco initialization ######
###### Pre-trained model f./exps/myexperiments_MIDDLE_0/mySSL_train_CHAOST2_Superpix_lbgroup0_scale_MIDDLE_vfold0_CHAOST2_Superpix_sets_0_1shot/default/snapshots/20000.pth has been loaded ######
INFO - main - ###### Load data ######
INFO - main - ###### Labels excluded in training : [2, 3] ######
INFO - main - ###### Unseen labels evaluated in testing: [2, 3] ######
###### Dataset: using external normalization statistics ######
###### index-by-class table has been reloaded ######
INFO - main - ###### Set validation nodes ######
INFO - main - ###### Starting validation ######
###### Using 1 shot evaluation!
###### Parts percentag